In [5]:
import PIL
import requests
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler


c:\Users\diosdadj\Anaconda3\envs\ai\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We propose a method for editing images from human instructions: given an input image and a written instruction that tells the model what to do, our model follows these instructions to edit the image. To obtain training data for this problem, we combine the knowledge of two large pretrained models---a language model (GPT-3) and a text-to-image model (Stable Diffusion)---to generate a large dataset of image editing examples. Our conditional diffusion model, InstructPix2Pix, is trained on our generated data, and generalizes to real images and user-written instructions at inference time. Since it performs edits in the forward pass and does not require per-example fine-tuning or inversion, our model edits images quickly, in a matter of seconds. We show compelling editing results for a diverse collection of input images and written instructions.

In [6]:
model_id = "timbrooks/instruct-pix2pix"

#pareceser que peta por el float26. Si lo llamo sin ese parametro fnciona

# pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None)
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, safety_checker=None)


pipe.enable_sequential_cpu_offload()
#pipe.to("cuda")
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

url = "https://raw.githubusercontent.com/timothybrooks/instruct-pix2pix/main/imgs/example.jpg"
def download_image(url):
    image = PIL.Image.open(requests.get(url, stream=True).raw)
    image = PIL.ImageOps.exif_transpose(image)
    image = image.convert("RGB")
    return image
image = download_image(url)


#load an image from disk


vae\diffusion_pytorch_model.safetensors not found


In [7]:
torch.cuda.empty_cache()

In [8]:

image = PIL.Image.open("jorge.jpg")

In [9]:
prompt = "color eyes in red"
images = pipe(prompt, image=image, num_inference_steps=10, image_guidance_scale=1).images[0]
#show the image in the notebook
images.show()


100%|██████████| 10/10 [00:35<00:00,  3.58s/it]


In [10]:
images.save(prompt[:10] + ".jpg")